# Warstwa gęsta

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Przygotowanie danych
X, y = load_iris(return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = tf.keras.utils.to_categorical(y, 3)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Budowa modelu
model = Sequential(
    [
        Input(shape=(4,)),
        Dense(10, activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ]
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Trenowanie z dynamicznym batch_size
for epoch in range(10):
    batch_size = np.random.choice([8, 16, 32, 64])
    print(f"Epoch {epoch + 1}: batch_size = {batch_size}")
    model.fit(X_train, y_train, batch_size=batch_size, epochs=1, verbose=0)

# Ewaluacja
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {acc:.3f}")


Epoch 1: batch_size = 8
Epoch 2: batch_size = 32
Epoch 3: batch_size = 8
Epoch 4: batch_size = 64
Epoch 5: batch_size = 64
Epoch 6: batch_size = 16
Epoch 7: batch_size = 32
Epoch 8: batch_size = 8
Epoch 9: batch_size = 16
Epoch 10: batch_size = 64
Test accuracy: 0.733


# Warstwa konwolucyjna

In [14]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    BatchNormalization,
    Dense,
)
from tensorflow.keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train[..., None] / 255.0
X_test = X_test[..., None] / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential(
    [
        Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
        BatchNormalization(),
        MaxPooling2D(),
        Conv2D(64, (3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(10, activation="softmax"),
    ]
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1)


c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
657/844 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9056 - loss: 0.3082

KeyboardInterrupt: 

# Warstwa rekurencyjna - LSTM + GRU (IMDB) 

In [6]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, GRU

max_features = 10000
max_len = 200

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

model = Sequential(
    [
        Embedding(max_features, 128),
        LSTM(64, return_sequences=True),
        GRU(32),
        Dense(1, activation="sigmoid"),
    ]
)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=2, batch_size=64, validation_split=0.1)


Epoch 1/2
352/352 ━━━━━━━━━━━━━━━━━━━━ 54s 140ms/step - accuracy: 0.7136 - loss: 0.5210 - val_accuracy: 0.8656 - val_loss: 0.3260
Epoch 2/2
352/352 ━━━━━━━━━━━━━━━━━━━━ 46s 129ms/step - accuracy: 0.9022 - loss: 0.2524 - val_accuracy: 0.8648 - val_loss: 0.3162


# Warstwa transformer - redukcja wymiarowości 

In [8]:
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Input, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf

# Wejście: sekwencja 50 kroków, każdy z 128 cech
inputs = Input(shape=(50, 128))

# Warstwa atencji
x = MultiHeadAttention(num_heads=4, key_dim=128)(inputs, inputs)
x = LayerNormalization()(x)

# Redukcja wymiarowości
x = Dense(64, activation="relu")(x)

# Redukcja po osi czasowej (średnia po krokach sekwencji)
x = Lambda(lambda t: tf.reduce_mean(t, axis=1))(x)

# Warstwa wyjściowa
x = Dense(1, activation="sigmoid")(x)

# Kompilacja modelu
model = Model(inputs, x)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 50, 128)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 50, 128)   │    263,808 │ input_layer_7[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 50, 128)   │        256 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 50, 64)    │      8,256 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 64)        │          0 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1)         │         65 │ lambda_2[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 272,385 (1.04 MB)

 Trainable params: 272,385 (1.04 MB)

 Non-trainable params: 0 (0.00 B)